In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io, transform
import torch
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets, models
from pathlib import Path

plt.ion()

In [2]:
class FaceDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.landmarks_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.landmarks_frame)
    
    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.landmarks_frame.iloc[idx, 0])
        image = io.imread(img_name)
        landmarks = self.landmarks_frame.iloc[idx, 1:].as_matrix()
        landmarks = landmarks.astype('float').reshape(-1, 2)
        sample = {'image': image, 'landmarks': landmarks}
        
        if self.transform:
            sample = self.transform(sample)
        return sample

In [3]:
data_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

face_dataset = datasets.ImageFolder(root='../data/train/',
                            transform=data_transform)

In [4]:
dataset_loader = torch.utils.data.DataLoader(face_dataset,
                                             batch_size=50, shuffle=True,
                                             num_workers=4)

In [5]:
epochs = 10
lr = 0.01
momentum = 0.5
seed = 1
log_interval = 50
torch.manual_seed(seed)
if torch.cuda.is_available(): # GPUが利用可能か確認
    device = 'cuda'
else:
    device = 'cpu'

In [6]:
model = models.resnet50(num_classes=2)
if device == 'cuda':
    model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [7]:
def train(model, device, train_loder, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loder):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx*len(data), len(train_loder.dataset), 100. * batch_idx / len(train_loder), loss.item()))

In [8]:
for epoch in range(1, epochs + 1):
        train(model, device, dataset_loader, optimizer, epoch)

Train Epoch: 1 [0/9565 (0%)]	Loss: 0.351854
Train Epoch: 1 [5000/9565 (52%)]	Loss: 0.011116
Train Epoch: 2 [0/9565 (0%)]	Loss: 0.181162
Train Epoch: 2 [5000/9565 (52%)]	Loss: 0.127001
Train Epoch: 3 [0/9565 (0%)]	Loss: 0.120994
Train Epoch: 3 [5000/9565 (52%)]	Loss: 0.095760
Train Epoch: 4 [0/9565 (0%)]	Loss: 0.089401
Train Epoch: 4 [5000/9565 (52%)]	Loss: 0.025563
Train Epoch: 5 [0/9565 (0%)]	Loss: 0.190442
Train Epoch: 5 [5000/9565 (52%)]	Loss: 0.029356
Train Epoch: 6 [0/9565 (0%)]	Loss: 0.057247
Train Epoch: 6 [5000/9565 (52%)]	Loss: 0.012467
Train Epoch: 7 [0/9565 (0%)]	Loss: 0.038670
Train Epoch: 7 [5000/9565 (52%)]	Loss: 0.059204
Train Epoch: 8 [0/9565 (0%)]	Loss: 0.025070
Train Epoch: 8 [5000/9565 (52%)]	Loss: 0.054790
Train Epoch: 9 [0/9565 (0%)]	Loss: 0.013472
Train Epoch: 9 [5000/9565 (52%)]	Loss: 0.038587
Train Epoch: 10 [0/9565 (0%)]	Loss: 0.068778
Train Epoch: 10 [5000/9565 (52%)]	Loss: 0.004765


In [9]:
torch.save(model.state_dict(), '../models/model2.pth')

In [7]:
model2 = models.resnet50(num_classes=2)
if device == 'cuda':
    model2 = torch.nn.DataParallel(model2, device_ids=range(torch.cuda.device_count()))
param = torch.load('../models/model2.pth')
model2.load_state_dict(param)

In [7]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataset_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [8]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [7]:
dataset_sizes = len(face_dataset)
class_names = face_dataset.classes

In [8]:
inputs, labels = iter(dataset_loader).next()

inputs = inputs.to(device)
labels = labels.to(device)
outputs = model2(inputs)
preds = torch.softmax(outputs, 1)

In [9]:
preds.size()

torch.Size([50, 2])

In [10]:
inputs.size()

torch.Size([50, 3, 224, 224])

In [11]:
preds

tensor([[3.7022e-09, 1.0000e+00],
        [5.0840e-04, 9.9949e-01],
        [1.3385e-05, 9.9999e-01],
        [5.9299e-06, 9.9999e-01],
        [1.4567e-06, 1.0000e+00],
        [1.5545e-05, 9.9998e-01],
        [7.8634e-01, 2.1366e-01],
        [3.5850e-07, 1.0000e+00],
        [2.9949e-05, 9.9997e-01],
        [4.7713e-06, 1.0000e+00],
        [5.9191e-02, 9.4081e-01],
        [3.7469e-03, 9.9625e-01],
        [1.9969e-07, 1.0000e+00],
        [2.7314e-06, 1.0000e+00],
        [5.6472e-11, 1.0000e+00],
        [1.0216e-04, 9.9990e-01],
        [1.7521e-05, 9.9998e-01],
        [8.4031e-08, 1.0000e+00],
        [6.6693e-08, 1.0000e+00],
        [6.7928e-07, 1.0000e+00],
        [6.4485e-07, 1.0000e+00],
        [7.0593e-11, 1.0000e+00],
        [6.3284e-10, 1.0000e+00],
        [2.0819e-05, 9.9998e-01],
        [3.3402e-06, 1.0000e+00],
        [5.8755e-04, 9.9941e-01],
        [1.6127e-04, 9.9984e-01],
        [3.1245e-06, 1.0000e+00],
        [7.1899e-03, 9.9281e-01],
        [4.115